In [20]:
import glob

raw_files = glob.glob('GermInnC/**/*.txt')

In [36]:
import os
import chardet
import magic 

data = []
error = 0

for file in raw_files:
    
    m_file = magic.Magic(mime_encoding=True)
    file_encoding = m_file.from_buffer(file)
    file = file.encode(file_encoding).decode('UTF-8')
    genre, period, region, year, title = os.path.basename(file).split('_')[:5]
    
    with open(file, 'rb') as f:
        text_bytes = f.read()
        encoding_info = chardet.detect(text_bytes)
        text = text_bytes.decode(encoding_info['encoding'])
        
    data.append({
        'text': text,
        'title': title,
        'genre': genre,
        'period': period,
        'region': region,
        'year': year
    })

print(f'Failed to read {error} files')

Failed to read 0 files


In [37]:
import pandas as pd

df = pd.DataFrame.from_dict(data)
df.to_csv('dataset.csv', encoding='UTF-8')

In [39]:
data[2]

{'text': '      Einleitung.\r\n      § 1.\r\n      Glaube und Arbeit.\r\n      Gewiß haben Weltkrieg und Revolution jede Art von Glauben erschüttert. Aber zerbrochen wurde doch nur der Glaube, der mit ungläubigen Gedanken über den Willen Gottes verbunden war. Wurde als dessen Ziel die Erhaltung der eignen Person des Glaubenden, die seines Volkes und die der ganzen Verfassung von Leben und Welt angesehen, so mußte der grausame Lauf der Ereignisse mit all diesen Werten auch den Glauben zu Fall bringen, der an ihnen hing. Wohl denen, die schon vorher an höhern und dauernden Gütern gehangen und Gottes Willen anders zu schauen gelernt hatten! Wohl auch noch den andern, die sich durch jenen Verlauf der Geschehnisse veranlaßt sahen, das Auge ihres Glaubens höher zu lenken, wo Gott selber und nicht der Abgott, das Geschöpf, das die Phantasie dem Wunsche geboren hat, thront und waltet! Zerbrechen kann nur ein Glaube, der da verlangt, daß sich Gott nach unserm Willen richtet, und sich weigert, s

In [81]:
tagged_files = glob.glob('GermInnC Tagged/**/*.txt')

In [125]:
import os

tagged_data = []
for file in tagged_files:
    genre, period, region, year, title = os.path.basename(file).split('_')[:5]
    tokens = []
    pos_tags = []
    lemmas = []
    with open(file, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.rstrip()
            try:
                token, pos_tag, lemma = line.split('\t')
            except ValueError:
                continue
            tokens.append(token)
            pos_tags.append(pos_tag)
            lemmas.append(lemma)
        
    tagged_data.append({
        'tokens': " ".join(tokens),
        'pos_tags': " ".join(pos_tags),
        'lemmas': " ".join(lemmas),
        'title': title,
        'genre': genre,
        'period': period,
        'region': region,
        'year': year
    })

In [126]:
tagged_data[0]

{'tokens': 'Lokales und Vermischtes . Der Kaiser in Breslau . Der Kaiser , der bekanntlich heute den Feierlichkeiten anläßlich der Vermählung der Gräfin Armgard zu Stolberg = Wernigerode mit dem Grafen Platen zu Hallermund in Landeshut beiwohnen wird , begibt sich gleich nach der Trauung nach Breslau , wo er heute Abend eintrifft . Der Kaiser wird sich , wie uns ein Privat = Telegramm aus Breslau meldet , sofort nach dem fürstbischhöflichen Palais begeben , um beim Kardinal Kopp das Souper einzunehmen . Die Rückreise des Kaisers nach Berlin erfolgt am Dienstag Nachmittag . – Die Kaiserin hat ihre Teilnahme an der Hochzeitsfeierlichkeit wegen leichter Unpäßlichkeit abgesagt . Gestern Vormittag wohnte der Kaiser dem Gottesdienst zu Potsdam bei und empfing um 12 ¾ Uhr den Staatsminister Budde . – Gestern Abend begab sich der Kaiser nach dem königlichen Schauspielhaus , um der Vorstellung von Blumenthals „ Mann wir altern “ und Rofstands „ Die Romantischen “ beizuwohnen . Im Schauspielhaus

In [127]:
import pandas as pd

df = pd.DataFrame.from_dict(tagged_data)
df.to_csv('tagged_dataset.csv', encoding='UTF-8')